# 🧪 Quantum Chemistry in Python (PySCF + Colab)

This notebook demonstrates simple quantum‑mechanical calculations in Google Colab using **PySCF**.


## ⚙️ 1. Installation
Run this once in Colab to install dependencies.

In [ ]:
!pip -q install pyscf py3Dmol


## 💧 2. Hartree–Fock on water
Build the molecule, run RHF, and report energy and HOMO–LUMO gap.

In [ ]:
import numpy as np
from pyscf import gto, scf

xyz = """
O  0.000000  0.000000  0.000000
H  0.000000  0.757160  0.586260
H  0.000000 -0.757160  0.586260
"""

mol = gto.Mole()
mol.build(atom=xyz, basis="def2-SVP", unit="Angstrom")

mf = scf.RHF(mol).run()

E_hf = mf.e_tot
mo_e = mf.mo_energy
occ = mf.mo_occ
homo = np.where(occ > 0)[0][-1]
lumo = homo + 1
gap_ev = (mo_e[lumo] - mo_e[homo]) * 27.211386

print(f"HF energy: {E_hf:.8f} Eh")
print(f"HOMO: {mo_e[homo]: .6f} Eh   LUMO: {mo_e[lumo]: .6f} Eh   Gap: {gap_ev:.2f} eV")


## 📊 3. Dipole, Mulliken charges, and 3D view


In [ ]:
from pyscf.prop import dipole as dip
from pyscf.scf import hf
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol
import numpy as np

# Dipole (Debye)
dm1 = mf.make_rdm1()
mu_au = dip.multipole(mol, dm1, unit='Debye')['total']
print(f"Dipole: {np.linalg.norm(mu_au):.2f} D  (components: {mu_au})")

# Mulliken population (atomic charges)
pop, chg = hf.mulliken_pop(mol, dm1, s=mol.intor('int1e_ovlp'))
print("Mulliken atomic charges:")
for sym, q in zip(mol.atom_symbols(), chg):
    print(f"  {sym:2s}: {q:+.3f}")

# 3D ball-and-stick view using RDKit + py3Dmol
rdmol = Chem.MolFromXYZBlock(xyz)
rdmol = Chem.AddHs(rdmol)
AllChem.EmbedMolecule(rdmol, AllChem.ETKDG())
mb = Chem.MolToMolBlock(rdmol)
view = py3Dmol.view(width=380, height=300)
view.addModel(mb, "mol")
view.setStyle({"stick": {}})
view.zoomTo()
view.show()


## 🌈 4. Optional: DFT (PBE0) and HOMO isosurface


In [ ]:
from pyscf import dft
from pyscf.tools import cubegen
import py3Dmol

mf_dft = dft.RKS(mol, xc="PBE0").run()
print(f"PBE0 energy: {mf_dft.e_tot:.8f} Eh")

# HOMO cube (may take ~10–30 s)
homo = np.where(mf_dft.mo_occ > 0)[0][-1]
cubegen.orbital(mol, "homo.cube", mf_dft.mo_coeff[:, homo], nx=40, ny=40, nz=40)

with open("homo.cube") as f:
    cube = f.read()

v = py3Dmol.view(width=420, height=340)
v.addVolumetricData(cube, "cube", {"isoval": 0.03, "color": "blue", "opacity": 0.6})
v.addVolumetricData(cube, "cube", {"isoval": -0.03, "color": "red", "opacity": 0.6})
v.addModel(mb, "mol")
v.setStyle({"stick": {}})
v.zoomTo()
v.show()
